In [2]:
import autograd.numpy as np
from autograd import grad, jacobian


inputs = np.array([[0.52, 1.12,  0.77],
                   [0.88, -1.08, 0.15],
                   [0.52, 0.06, -1.30],
                   [0.74, -2.49, 1.39]])
targets = np.array([True, True, False, True])

weights = np.random.rand(3)

def sigmoid(x):
    return 0.5 * (np.tanh(x / 2.) + 1)

def predict(inputs, weights):
    return sigmoid(inputs @ weights)

def logistic_loss(weights):
    preds = predict(inputs, weights)
    label_probs = preds * targets + (1 - preds) * (1 - targets)
    return -np.sum(np.log(label_probs))

loss_gradient = grad(logistic_loss)
for i in range(100):
    weights -= 0.1 * loss_gradient(weights)


print(f'W: {weights}, loss: {logistic_loss(weights)}')

W: [ 1.98762096 -0.47163551  3.1932936 ], loss: 0.15600271577914385


In [3]:
from numpy.typing import NDArray
from __future__ import annotations
from abc import ABC, abstractmethod
import operator as op

class Node:
    def __init__(self, value: NDArray, parents: list[Node], primitive: Primitive) -> None:
        self.value = value
        self._parents = parents
        self._primitive = primitive


class Primitive(ABC):
    def __init__(self, fun) -> None:
        self._fun = fun
    
        
    def __call__(self, *args: Node):
        values = [n.value for n in args]
        result = self._fun(*values)
        return Node(result, args, self)
    
    @abstractmethod
    def grad(self, argnum):
        ...

        
class sum(Primitive):
    def __init__(self) -> None:
        super().__init__(op.add)

    def grad(self, argnum):
        def gradient(self, a: NDArray, b: NDArray):
            return np.ones_like([a, b][argnum])
        return gradient
    
class mult(Primitive):
    def __init__(self) -> None:
        super().__init__(op.mul)

    def grad(self, argnum):
        def gradient(self, a: NDArray, b: NDArray):
            # ?
            return [a, b][argnum - 1]
        return gradient
    
class dibision(Primitive):
    def __init__(self) -> None:
        super().__init__(op.truediv)
    
    def grad(self):
        def gradient(a: NDArray, b: NDArray):
            return [
                1 / b,
                (b - a) / np.power(a, b)
                ]
        return gradient 
    
class matmul(Primitive):
    def __init__(self) -> None:
        super().__init__(op.matmul)
    
    def grad(self):
        def gradient(a, b):
            return (a @ b)
        return gradient

In [26]:
def test(X: NDArray, Y: NDArray):
    A = np.array([1, 2, 3], dtype=np.float64)
    B = np.array([4, 5, 6], dtype=np.float64)
    return (A + B) @ X

X = np.array([2,2,2], dtype=np.float64)
Y = np.array([1,1,1], dtype=np.float64)
test_grad = grad(test, 0)
print(test(X, Y), test_grad(X, Y))
# Actually, expected. Now, let's reproduce it with my great lib



42.0 [5. 7. 9.]


In [5]:
def test(X: NDArray):
    XNode = Node(X, [], None)
    A = Node(np.array([1, 2, 3], dtype=np.float64), [], None)
    B = Node(np.array([4, 5, 6], dtype=np.float64), [], None)
    return matmul()(sum()(A, B), XNode)



print(test(np.array([2,2,2])).value, test_grad(np.array([2,2,2])))

42.0 [5. 7. 9.]


In [4]:
import autograd.numpy as anp
import numpy as np


anp.array([1,2,3]) + np.array([3,4,5])

ModuleNotFoundError: No module named 'autograd.numpy'; 'autograd' is not a package